In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Draggable Database Interface</title>
  <link rel="stylesheet" href="https://code.jquery.com/ui/1.13.3/themes/base/jquery-ui.css">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.1.1/css/all.min.css">
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }
    .container {
      position: absolute;
      top: 0;
      left: 0;
      width: 100vw;
      height: 100vh;
      background: #f4f4f4;
      overflow: auto;
      padding: 20px;
      box-sizing: border-box;
    }
    .table-box {
      width: 300px;
      max-height: 400px;
      padding: 10px;
      background: #fff;
      border: 1px solid #ccc;
      border-radius: 5px;
      box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
      position: absolute;
      overflow: hidden;
      display: flex;
      flex-direction: column;
    }
    .table-box h2 {
      margin: 0;
      font-size: 18px;
      background: #f4f4f4;
      padding: 5px;
      border-radius: 5px;
      cursor: move;
      display: flex;
      justify-content: space-between;
      align-items: center;
    }
    .table-box h2 .delete-table-btn {
      color: red;
      cursor: pointer;
      margin-left: 10px;
    }
    .table-container {
      flex: 1;
      max-height: 300px;
      overflow-y: auto;
      margin-top: 10px;
    }
    .table-box table {
      width: 100%;
      border-collapse: collapse;
    }
    .table-box th, .table-box td {
      padding: 5px;
      text-align: left;
      border: 1px solid #ddd;
    }
    .delete-column-btn {
      color: red;
      cursor: pointer;
      margin-left: 5px;
    }
    .add-table-btn {
      margin: 10px;
      padding: 10px;
      background: #333;
      color: #fff;
      text-decoration: none;
      border-radius: 5px;
      cursor: pointer;
    }
    .add-table-btn:hover {
      background: #555;
    }
    .line {
      position: absolute;
      background: #000;
    }
    .toggle-btn {
      cursor: pointer;
      font-size: 18px;
    }
    .add-column-btn {
      margin: 10px;
      padding: 5px;
      background: #28a745;
      color: #fff;
      text-decoration: none;
      border-radius: 5px;
      cursor: pointer;
      display: block;
      text-align: center;
    }
    .add-column-btn:hover {
      background: #218838;
    }
    .fa {
      font-size: 14px;
    }
    .edit-joins-btn {
      margin: 10px;
      padding: 5px;
      background: #007bff;
      color: #fff;
      text-decoration: none;
      border-radius: 5px;
      cursor: pointer;
      display: block;
      text-align: center;
    }
    .edit-joins-btn:hover {
      background: #0056b3;
    }
    .dialog {
      display: none;
      position: fixed;
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%);
      background: #fff;
      border: 1px solid #ccc;
      border-radius: 5px;
      padding: 20px;
      box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
      z-index: 1000;
    }
    .dialog h3 {
      margin: 0;
      font-size: 18px;
    }
    .dialog input, .dialog textarea {
      width: 100%;
      padding: 10px;
      margin: 5px 0;
      border: 1px solid #ccc;
      border-radius: 5px;
    }
    .dialog button {
      margin-top: 10px;
      padding: 10px;
      background: #007bff;
      color: #fff;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }
    .dialog button:hover {
      background: #0056b3;
    }
  </style>
  <script src="https://code.jquery.com/jquery-3.7.1.js"></script>
  <script src="https://code.jquery.com/ui/1.13.3/jquery-ui.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/leader-line"></script>
</head>
<body>
  <div class="container" id="container">
    <a class="add-table-btn" id="add-table-btn">Add Table</a>
    
  </div>
  <script>
    let tableCount = 0;
    const container = document.getElementById('container');
    const lines = [];
    let joinsData = {};
    let currentJoin = null;

    $(function() {
      // Load JSON files
      Promise.all([
        fetch('static/database_details.json').then(response => response.json()),
        fetch('static/database_joins.json').then(response => response.json())
      ])
      .then(([tableData, loadedJoinsData]) => {
        joinsData = loadedJoinsData;
        tableData.Tables.forEach(table => addTable(table));
        // Ensure tables are added before drawing lines
        setTimeout(() => {
          drawLines();
        }, 500); // Adjust timeout as needed
      })
      .catch(error => console.error('Error loading JSON data:', error));

      $('#add-table-btn').on('click', function() {
        addTable();
      });

      $('#container').on('scroll', updateLines);

    
    });

    function addTable(tableData) {
      const tableName = tableData ? tableData.name : 'Table ' + (tableCount + 1);

      // Check if the table already exists
      if (document.getElementById(tableName)) {
        console.warn(`Table ${tableName} already exists!`);
        return;
      }

      tableCount++;
      const tableBox = document.createElement('div');
      tableBox.className = 'table-box ui-draggable ui-resizable';
      tableBox.id = tableName;

      // Add random position
      tableBox.style.top = (Math.random() * (window.innerHeight - 400)) + 'px';
      tableBox.style.left = (Math.random() * (window.innerWidth - 300)) + 'px';

      const primaryKeyColumn = tableData ? tableData.columns.find(col => col.constraints && col.constraints.includes("PRIMARY KEY")) : null;

      tableBox.innerHTML = `
        <h2>
          <span class="table-name" contenteditable="false">${tableName}</span>
          <span class="toggle-btn">&#9660;</span>
          <span class="delete-table-btn"><i class="fa fa-trash"></i></span>
          <span class="edit-joins-btn">Edit Joins</span>
        </h2>
        <div class="table-container">
          <table>
            <thead>
              <tr>
                <th>Column Name</th>
                ${primaryKeyColumn ? '<th>Primary Key</th>' : ''}
                <th>Action</th>
              </tr>
            </thead>
            <tbody id="table-${tableName}-columns">
              ${tableData ? tableData.columns.map(column => `
                <tr>
                  <td>${column.name}</td>
                  ${primaryKeyColumn ? (primaryKeyColumn.name === column.name ? '<td>✔</td>' : '') : ''}
                  <td><span class="delete-column-btn"><i class="fa fa-trash"></i></span></td>
                </tr>
              `).join('') : ''}
            </tbody>
          </table>
          <a class="add-column-btn" data-table="${tableName}">Add Column</a>
        </div>
      `;

      container.appendChild(tableBox);

      $(tableBox).draggable({
        containment: 'parent'
      }).resizable({
        containment: 'parent'
      });

      // Add event listeners
      $(tableBox).find('.delete-table-btn').on('click', function() {
        $(tableBox).remove();
        removeLinesForTable(tableName);
      });

      $(tableBox).find('.add-column-btn').on('click', function() {
        const tableName = $(this).data('table');
        const columnName = prompt('Enter new column name:');
        if (columnName) {
          addColumnToTable(tableName, columnName);
        }
      });

      $(tableBox).find('.delete-column-btn').on('click', function() {
        $(this).closest('tr').remove();
      });

      $(tableBox).find('.edit-joins-btn').on('click', function() {
        openEditJoinsDialog(tableName);
      });

      // Add a toggle feature to hide/show columns
      $(tableBox).find('.toggle-btn').on('click', function() {
        $(tableBox).find('.table-container').toggle();
        const isExpanded = $(tableBox).find('.table-container').is(':visible');
        $(this).html(isExpanded ? '&#9660;' : '&#9654;');
      });
    }

    function addColumnToTable(tableName, columnName) {
      const table = document.getElementById(tableName);
      if (table) {
        const columnTableBody = document.getElementById(`table-${tableName}-columns`);
        const newRow = document.createElement('tr');
        newRow.innerHTML = `
          <td>${columnName}</td>
          <td><span class="delete-column-btn"><i class="fa fa-trash"></i></span></td>
        `;
        columnTableBody.appendChild(newRow);

        $(newRow).find('.delete-column-btn').on('click', function() {
          $(this).closest('tr').remove();
        });
      }
    }

    function openEditJoinsDialog(tableName) {
      // Find existing joins for this table
      const tableJoins = joinsData.Joins.filter(join => join.models.includes(tableName));

      if (tableJoins.length > 0) {
        // For simplicity, edit the first join related to this table
        currentJoin = tableJoins[0];
        $('#join-name').val(currentJoin.name);
        $('#join-type').val(currentJoin.joinType);
        $('#join-condition').val(currentJoin.condition);
        $('#edit-joins-dialog').show();
      } else {
        alert('No joins found for this table.');
      }
    }

    function drawLines() {
      removeLines();
      joinsData.Joins.forEach(join => {
        const [sourceTable, targetTable] = join.models;
        const sourceElem = document.getElementById(sourceTable);
        const targetElem = document.getElementById(targetTable);
        if (sourceElem && targetElem) {
          const sourcePos = getPosition(sourceElem);
          const targetPos = getPosition(targetElem);
          const line = new LeaderLine(
            LeaderLine.pointAnchor(sourceElem.querySelector('h2')),
            LeaderLine.pointAnchor(targetElem.querySelector('h2')),
            {
              color: '#000',
              size: 3,
              path: 'smooth'
            }
          );
          lines.push(line);
        }
      });
    }

    function removeLines() {
      lines.forEach(line => line.remove());
      lines.length = 0;
    }

    function removeLinesForTable(tableName) {
      lines.forEach(line => {
        if (line._points[0].element.id === tableName || line._points[1].element.id === tableName) {
          line.remove();
        }
      });
    }

    function updateLines() {
      removeLines();
      drawLines();
    }

    function getPosition(elem) {
      const rect = elem.getBoundingClientRect();
      return {
        x: rect.left + window.scrollX + rect.width / 2,
        y: rect.top + window.scrollY + rect.height / 2
      };
    }
  </script>
</body>
</html>


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX
from examples import get_example_selector
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from get_tables import FunctionalTools
from get_llm import get_llm_sql
from get_memory import get_memory
from config_variables import DB_USER, DB_PASSWORD, DB_HOST, DB_NAME

memory = get_memory()
llm = get_llm_sql()

In [ ]:
user_query="no of work which are currently active."

table_recommendation_prompt = PromptTemplate(
            input_variables=["prompts","memory","user_query"],
            template="""
            You are an assistant that recommends relevant database tables for SQL queries.

            Return the names of ALL the SQL tables only from given table descriptions and no need to assume any table that MIGHT be relevant to the user question. \
            The tables are:
            {prompts} and
            History: {memory}

            User input: {user_query}

            Answer only with table names in comma-separated format.
            """
        )

recommendation_chain = LLMChain(llm=llm, prompt=table_recommendation_prompt, output_key="table_list")
recommendations = recommendation_chain.run(prompts=prompts,memory=memory,user_query=user_query)

In [ ]:
prompts

In [ ]:
recommendations

In [ ]:
recommended_tables = [name.strip() for name in recommendations.split(',')]
recommended_tables

In [ ]:
import json
from sentence_transformers import SentenceTransformer

# Load the JSON file (assumed to be not excessively large for simplicity)

l=["work_exp_temps","work_dashboards"]


# Assuming you have a function to send the prompt to Gemini API
# send_to_gemini_api(prompt)


In [ ]:
file=json.load(open("config.json"))
print(file)

In [ ]:
document.addEventListener('DOMContentLoaded', () => {
    const sendButton = document.getElementById('send-button');
    const userInput = document.getElementById('user-input');
    const chatContainer = document.getElementById('chat-container');
    const summaryButton = document.getElementById('summary-button');
    const summaryModal = document.getElementById('summary-modal');
    const closeButton = document.querySelector('.close-button');
    const summaryText = document.getElementById('summary-text');
    const updateFilesButton = document.getElementById('update-files-button');
    const fileNameInput = document.getElementById('file-name');
    const fileContentInput = document.getElementById('file-content');
    const newChatButton = document.getElementById('new-chat-button');
    const chatList = document.getElementById('chat-list');

    sendButton.addEventListener('click', () => {
        const userInputValue = userInput.value;
        if (userInputValue.trim()) {
            fetch('/chat', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ user_input: userInputValue })
            })
            .then(response => response.json())
            .then(message => {
                const userMessage = document.createElement('div');
                userMessage.classList.add('chat-message', 'user-message');
                userMessage.setAttribute("data-key", message.key);
                userMessage.setAttribute("data-chat-id", message.chat_id);
                userMessage.innerHTML = `<p class="user-input">${message.user_input}</p>`;
                chatContainer.appendChild(userMessage);

                const assistantMessage = document.createElement('div');
                assistantMessage.classList.add('chat-message', 'assistant-message');
                assistantMessage.setAttribute("data-key", message.key);
                assistantMessage.setAttribute("data-chat-id", message.chat_id);
                assistantMessage.innerHTML = `
                    <p>${message.result.answer}</p>
                    <textarea class="sql-query">${message.result.sql_query}</textarea>
                    <button class="save-button" data-key="${message.key}" data-chat-id="${message.chat_id}">Save</button>
                    <button class="run-button" data-key="${message.key}" data-chat-id="${message.chat_id}">Run Query</button>
                    <div class="query-result-container" style="max-height: 200px; overflow-y: auto;"></div>
                    <button class="open-window-button" data-key="${message.key}" data-chat-id="${message.chat_id}">Open in Separate Window</button>`;
                chatContainer.appendChild(assistantMessage);
    
                userInput.value = '';
            });
        }
    });

    newChatButton.addEventListener('click', () => {
        fetch('/new_chat', {
            method: 'POST',
        })
        .then(response => response.json())
        .then(data => {
            const newChatItem = document.createElement('li');
            newChatItem.setAttribute('data-chat-id', data.chat_id);
            newChatItem.textContent = 'New Chat';
            chatList.appendChild(newChatItem);

            newChatItem.addEventListener('click', () => {
                switchChatSession(data.chat_id);
            });

            // Clear the chat container
            chatContainer.innerHTML = '';
        });
    });

    const switchChatSession = (chat_id) => {
        fetch('/switch_chat', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/x-www-form-urlencoded',
            },
            body: new URLSearchParams({ chat_id })
        })
        .then(response => response.json())
        .then(data => {
            chatContainer.innerHTML = '';
            data.messages.forEach(message => {
                const userMessage = document.createElement('div');
                userMessage.classList.add('chat-message', 'user-message');
                userMessage.setAttribute("data-key", message.key);
                userMessage.setAttribute("data-chat-id", chat_id);
                userMessage.innerHTML = `<p>${message.user_input}</p>`;
                chatContainer.appendChild(userMessage);

                const assistantMessage = document.createElement('div');
                assistantMessage.classList.add('chat-message', 'assistant-message');
                assistantMessage.setAttribute("data-key", message.key);
                assistantMessage.setAttribute("data-chat-id", chat_id);
                assistantMessage.innerHTML = `
                    <p>${message.result.answer}</p>
                    <textarea class="sql-query">${message.result.sql_query}</textarea>
                    <button class="save-button" data-key="${message.key}" data-chat-id="${chat_id}">Save</button>
                    <button class="run-button" data-key="${message.key}" data-chat-id="${chat_id}">Run Query</button>
                    <div class="query-result-container" style="max-height: 200px; overflow-y: auto;"></div>
                    <button class="open-window-button" data-key="${message.key}" data-chat-id="${chat_id}">Open in Separate Window</button>`;
                chatContainer.appendChild(assistantMessage);
            });
        });
    };

    chatList.addEventListener('click', (event) => {
        const chatId = event.target.getAttribute('data-chat-id');
        if (chatId) {
            switchChatSession(chatId);
        }
    });

    summaryButton.addEventListener('click', () => {
        fetch('/summary')
            .then(response => response.json())
            .then(data => {
                summaryText.textContent = data.chat_history.join('\n');
                summaryModal.style.display = 'block';
            });
    });

    closeButton.addEventListener('click', () => {
        summaryModal.style.display = 'none';
    });

    window.addEventListener('click', (event) => {
        if (event.target == summaryModal) {
            summaryModal.style.display = 'none';
        }
    });

    document.addEventListener('click', (event) => {
        const key = event.target.getAttribute('data-key');
        const chatId = event.target.getAttribute('data-chat-id');
        if (!key || !chatId) return; // exit if key or chat_id is null

        if (event.target.classList.contains('save-button')) {
            const userQuery = document.querySelector(`.user-message[data-key="${key}"][data-chat-id="${chatId}"] .user-input`).textContent;
            const sqlQuery = document.querySelector(`.assistant-message[data-key="${key}"][data-chat-id="${chatId}"] .sql-query`).value;
            fetch('/update_fewshot', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/x-www-form-urlencoded',
                },
                body: new URLSearchParams({ user_query: userQuery, sql_query: sqlQuery })
            })
            .then(response => {
                if (!response.ok) {
                    throw new Error('Network response was not ok');
                }
                alert('Query saved successfully');
            })
            .catch(error => {
                console.error('There was a problem with the fetch operation:', error);
            });
        }

        if (event.target.classList.contains('run-button')) {
            const sqlQuery = document.querySelector(`.assistant-message[data-key="${key}"][data-chat-id="${chatId}"] .sql-query`).value;
            const queryResultContainer = document.querySelector(`.assistant-message[data-key="${key}"][data-chat-id="${chatId}"] .query-result-container`);
            fetch('/run_query', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/x-www-form-urlencoded',
                },
                body: new URLSearchParams({ chat_id: chatId, key: key, sql_query: sqlQuery })
            })
            .then(response => response.json())
            .then(result => {
                queryResultContainer.innerHTML = `<pre>${JSON.stringify(result, null, 2)}</pre>`;
            });
        }

        if (event.target.classList.contains('open-window-button')) {
            const sqlQuery = document.querySelector(`.assistant-message[data-key="${key}"][data-chat-id="${chatId}"] .sql-query`).value;
            window.open(`/run_query_window?chat_id=${encodeURIComponent(chatId)}&key=${encodeURIComponent(key)}&sql_query=${encodeURIComponent(sqlQuery)}`);
        }
    });

    updateFilesButton.addEventListener('click', () => {
        const fileName = fileNameInput.value;
        fetch(`/get_file_content/${fileName}`)
        .then(response => response.text())
        .then(content => {
            fileContentInput.value = content;
            updateFilesModal.style.display = 'block';
        });
    });

    document.querySelector('#save-file-button').addEventListener('click', () => {
        const fileName = fileNameInput.value;
        const fileContent = fileContentInput.value;
        fetch('/save_file', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/x-www-form-urlencoded',
            },
            body: new URLSearchParams({ file_name: fileName, file_content: fileContent })
        })
        .then(response => {
            if (!response.ok) {
                throw new Error('Network response was not ok');
            }
            alert('File saved successfully');
            updateFilesModal.style.display = 'none';
        })
        .catch(error => {
            console.error('There was a problem with the fetch operation:', error);
        });
    });

    const updateFilesModal = document.getElementById('update-files-modal');
    const closeButtonUpdateFiles = document.querySelector('.close-button');
    closeButtonUpdateFiles.addEventListener('click', () => {
        updateFilesModal.style.display = 'none';
    });

    window.addEventListener('click', (event) => {
        if (event.target == updateFilesModal) {
            updateFilesModal.style.display = 'none';
        }
    });
});





